#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report,f1_score, confusion_matrix, accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LogisticRegression_base.csv')
df.head()

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df_train_window = df.tail(127)
df_train_window

In [ ]:
# สร้าง df_train_window['is_hotseller'] โดย แบ่งข้อมูลด้วย ควอนไทล์ (Quantiles)
#                       - ยอดขายที่อยู่ในควอนไทล์ล่างสุดน้อยกว่าหรือเท่ากับ 33% เป็น Class 0 (เงียบ)
#                       - ยอดขายที่อยู่ระหว่าง มากกว่า33%-น้อยกว่า66% เป็น Class 1 (ปกติ)
#                       - ยอดขายที่อยู่ในควอนไทล์สูงสุด เท่ากับหรือมากกว่า66% เป็น Class 2 (ขายดี)

# Calculate the quantiles
quantile_33 = df_train_window['sales_sum'].quantile(0.33)
quantile_66 = df_train_window['sales_sum'].quantile(0.66)

# Create the 'is_hotseller' column based on quantiles
df_train_window['is_hotseller'] = pd.cut(df_train_window['sales_sum'],
                                        bins=[-float('inf'), quantile_33, quantile_66, float('inf')],
                                        labels=[0, 1, 2],
                                        right=True)

# Convert to integer type
df_train_window['is_hotseller'] = df_train_window['is_hotseller'].astype(int)

print(df_train_window.head())
print(df_train_window['is_hotseller'].value_counts())


In [ ]:
# นับจำนวน is_hotseller ที่เป็น 0, 1, 2

print("\nNumber of is_hotseller values:")
print(df_train_window['is_hotseller'].value_counts().sort_index())

In [ ]:
df_train_window.tail(8)

In [ ]:
df_train_window = df_train_window.reindex(columns=['is_hotseller', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'])

In [ ]:
test_data = df_train_window.tail(7)
train_data = df_train_window.head(120)

print("Test Data:")
print(test_data)

print("\nTrain Data:")
print(train_data)

In [ ]:
X = train_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']]
y = train_data['is_hotseller']

#Train

มีความจำเป็นต้องกำหนด class_weight='balanced' เนื่องจากปัญหา Imbalance target ซึ่งเกิดจากการต้องการแยกวันสำคัญ ซึ่งคือวันขายดีกว่าปกติ ซึ่งเป็นส่วนน้อย

In [ ]:
# เพิ่ม class_weight='balanced' เข้าไป
#model = LogisticRegression(class_weight='balanced')
model = LogisticRegression()
model.fit(X, y)

In [ ]:
X_test = test_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']]
predictions = model.predict(X_test)

# Print the predictions
print("\nPredictions:")
predictions

#Evaluate

In [ ]:
y_test = test_data['is_hotseller']

In [ ]:
# คำนวณ Confusion Matrix
cm = confusion_matrix(y_test, predictions)

# แสดง Confusion Matrix
print("\nConfusion Matrix:")
print(cm)

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, predictions))